# 1. Preprocessing
## Import data

In [3]:
import pandas as pd
df = pd.read_csv('training.300000.processed.noemoticon.csv', encoding='ISO-8859-1')

## Text clean

In [8]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to C:\Users\Liu
[nltk_data]     Yifan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Liu
[nltk_data]     Yifan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))


def clean_text(text):
    # transfer to lower characters
    text = text.lower()
    
    # eliminate url link
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # eliminate special character
    text = re.sub(r'@\w+|#\w+', '', text)
    
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # eliminate stop word
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    
   
    return ' '.join(tokens)

# apply clean function to data frame
df['cleaned_text'] = df['text'].apply(clean_text)

df[['text', 'cleaned_text']].head()

,text,cleaned_text
0,@chrishasboobs AHHH I HOPE YOUR OK!!!,ahhh hope ok
1,"@misstoriblack cool , i have no tweet apps fo...",cool tweet apps razr
2,@TiannaChaos i know just family drama. its la...,know family drama lamehey next time u hang kim...
3,School email won't open and I have geography ...,school email wont open geography stuff revise ...
4,upper airways problem,upper airways problem


## Tokenization, Padding

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Define tokenizer
tokenizer = Tokenizer(num_words=10000)


tokenizer.fit_on_texts(df['cleaned_text'])

# Transfer to sequences
sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

# Create index of each tokens
word_index = tokenizer.word_index

# Padding sequence to a constant length
X = pad_sequences(sequences, maxlen=10000)

# Transfer original labels to 0-1 label
y = df['sentiment'].values  
y = (y == 4).astype(int) 


## Divide dataset into two parts, trainset and testset

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Deep learning

# CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Embedding, Dropout, Flatten


# 定义CNN模型
model = Sequential()

# Embedding 层：自动学习词嵌入，input_dim 是词汇表大小，output_dim 是词嵌入的维度
model.add(Embedding(input_dim=10000, output_dim=100, input_length=X_train.shape[1]))

# 卷积层：128个滤波器，卷积核大小为5
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))

# 最大池化层
model.add(MaxPooling1D(pool_size=2))

# 扁平化
model.add(Flatten())

# Dropout层，防止过拟合
model.add(Dropout(0.5))

# 全连接层，64个神经元
model.add(Dense(64, activation='relu'))

# 输出层：二分类任务，使用sigmoid激活函数
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 输出模型结构
model.summary()

history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)